In [4]:
import sys
import os
import json
import logging
import csv
from datetime import date
import pandas as pd

In [5]:
current_folder = globals()['_dh'][0]
project_root = os.path.abspath(os.path.join(current_folder, '..'))

# Add the project root directory to the Python path
sys.path.insert(-1,project_root)

from admin_classes import prodAdmin, betaAdmin, testAdmin

In [6]:
# Set up logging
logging.basicConfig(level=logging.ERROR, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

In [6]:
class WaterSchema:
    access: str
    address: str
    city: str
    description: str
    filtration: str
    gp_id: str
    handicap: str
    hours: list
    lat: float
    lon: float
    norms_rules: str
    organization: str
    permanently_closed: bool
    phone: str
    quality: str
    service: str
    statement: str
    status: str
    tap_type: str
    tapnum: int
    vessel: str
    zip_code: str

# Helper Functions

In [8]:
def validate_json(data, error_count, warning_count, unexpected_count):
    ''' Validates a JSON object against a schema
        Returns the number of errors, warnings, and unexpected types found
    '''
    expected_data_types = WaterSchema.__annotations__

    for key, value in data.items():
        if key in expected_data_types:
            if not isinstance(value, expected_data_types[key]):
                logger.error(f"Invalid data type for key '{key}': Expected {expected_data_types[key]}, got {type(value)}")
                error_count += 1
        elif key not in expected_data_types:
            logger.error(f"Unexpected key '{key}'")
            error_count += 1
        else:
            logger.warning(f"Unexpected key '{key}'")
            warning_count += 1

        if isinstance(value, dict):
            error_count, warning_count, unexpected_count = validate_json(value, error_count, warning_count, unexpected_count)
        elif isinstance(value, list):
            for item in value:
                if isinstance(item, dict):
                    error_count, warning_count, unexpected_count = validate_json(item, error_count, warning_count, unexpected_count)
                else:
                    unexpected_count += 1

    return error_count, warning_count, unexpected_count

In [7]:
def validate_data(admin_class, db, db_name):
    ''' Retrieves data from a database and validates it against a schema 
        Saves the results to a csv file
    '''
    data = admin_class().getDb(db)
    print(f"\nData retrieved from {db}")
    print(f"\n\tLength of dataset: {len(data)}")

    # Validate data
    error_count = 0
    warning_count = 0
    unexpected_count = 0
    for x in data:
        e, w, u = validate_json(x, error_count, warning_count, unexpected_count)
        error_count += e
        warning_count += w
        unexpected_count += u

    logging.info(f"Errors: {error_count}")
    logging.info(f"Warnings: {warning_count}")
    logging.info(f"Unexpected Types: {unexpected_count}")

    # Create dataframe with results of validation
    df_results = pd.DataFrame({
        "Database": [db_name],
        "Errors": [error_count],
        "Warnings": [warning_count],
        "Unexpected Types": [unexpected_count]}) 

    logging.info(f"Length of dataset: {len(data)}")

    # Write report of all the logging with total counts
    today = date.today()
    csv_file_path = f"{today.strftime('%Y-%m-%d')}_firebase_db_validation_{db_name}.csv"

    # check if csv file exists and if so delete it
    if os.path.exists(csv_file_path):
        os.remove(csv_file_path)

    # save as csv and text file dated with today's date
    df_results.to_csv(csv_file_path, index=False)
    logging.info(f"Validation results saved to {csv_file_path}")

# Main Function

In [7]:
# Initialize the prod, beta, and test environment database references.
logging.info("Initializing database references...")
water_prod = prodAdmin().water_db_live
water_beta = betaAdmin().water_db_live
water_test = testAdmin().water_db_live

In [8]:
# get dataframes for each database
data_prod = prodAdmin().getDb(water_prod)

# pop the first item from the list (author entry)
data_prod.pop(0)

# create dataframe from data
df_prod = pd.DataFrame(data_prod)

In [34]:
df_prod.head()

,access,address,city,description,filtration,gp_id,handicap,hours,lat,lon,...,location_verified,outdoor_indoor,image,special,active,direction,ill,impressions,tab_id,unit
0,Public,1020 Lombard St.,Philadelphia,Drinking fountain located within the Recreatio...,No,ChIJocPgsybGxokR3QyYHHmQ118,Unsure,"[{'close': {'day': 0, 'time': '2100'}, 'open':...",39.943787,-75.159048,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Public,Market St. between 10th & 12th Sts.,Philadelphia,Drinking fountain near SEPTA ticket window at ...,No,ChIJSVdgvSnGxokRWJqLCbC4xzQ,Yes,NaN,39.952500,-75.158056,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Private,1500 Chestnut St.,Philadelphia,Drinking fountain located near bathrooms at th...,Unsure,ChIJCSet3i_GxokRecQeyoGrAl0,Unsure,"[{'close': {'day': 0, 'time': '1700'}, 'open':...",39.950861,-75.165866,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Public,16th St. & JFK Blvd.,Philadelphia,Drinking fountain located behind the SEPTA tic...,No,ChIJ9Sdt-zHGxokRad5acsk-ifo,Yes,NaN,39.954101,-75.166818,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Public,1901 Vine St.,Philadelphia,"Drinking fountain on 1st floor, outside of Gov...",No,ChIJ_x-wODPGxokREDE-Lq4X9dE,Unsure,"[{'close': {'day': 1, 'time': '1700'}, 'open':...",39.959269,-75.170716,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
import pandera as pa
from pandera import Column, Check
from altered_schema import schema

ModuleNotFoundError: No module named 'altered_schema'

In [9]:
# infer schema from dataframe
water_prod_schema = pa.infer_schema(df_prod)

In [15]:
with open('water_prod_schema.py', 'w') as f:
    f.write(water_prod_schema.to_script())

In [31]:
from water_schema_altered import schema

In [33]:
try: 
    water_prod_schema.validate(df_prod, lazy = True)
except pa.errors.SchemaErrors as err:
    print("Schema errors and failure cases:")
    display(err.failure_cases)
    print("\nDataFrame object that failed validation:")
    display(err.data)

c:\Users\keonm\miniconda3\envs\phlask_dev\Lib\site-packages\pandera\api\pandas\container.py:362: UserWarning: This <class 'pandera.api.pandas.container.DataFrameSchema'> is an inferred schema that hasn't been modified. It's recommended that you refine the schema by calling `add_columns`, `remove_columns`, or `update_columns` before using it to validate data.
  warnings.warn(


# OLD - 

In [10]:
# Validate data in each database - WATER TAPS DATA
for i, (db, db_name) in enumerate([(water_prod, 'water_prod'), (water_beta, 'water_beta'), (water_test, 'water_test')]):
    logging.info(f"\n\t{i + 1}. Validating data in {db_name} database...")
    
    if i == 0:
        admin_class = prodAdmin
    elif i == 1:
        admin_class = betaAdmin
    else:
        admin_class = testAdmin

    # Retrieve data from database
    data = admin_class().getDb(db)
    logging.info(f"\nData retrieved from {db}")
    logging.info(f"\n\tLength of dataset: {len(data)}")

    # validate against schema - get total errors, warnings, and unexpected types
    validate_data(admin_class, db, db_name)
    break

2023-04-30 08:55:20,193 - ERROR - Unexpected key 'author'
2023-04-30 08:55:20,194 - ERROR - Unexpected key 'datetime'
2023-04-30 08:55:20,195 - ERROR - Unexpected key 'purpose'
2023-04-30 08:55:20,196 - ERROR - Unexpected key 'close'
2023-04-30 08:55:20,197 - ERROR - Unexpected key 'day'
2023-04-30 08:55:20,198 - ERROR - Unexpected key 'time'
2023-04-30 08:55:20,198 - ERROR - Unexpected key 'open'
2023-04-30 08:55:20,199 - ERROR - Unexpected key 'day'
2023-04-30 08:55:20,199 - ERROR - Unexpected key 'time'
2023-04-30 08:55:20,201 - ERROR - Unexpected key 'close'
2023-04-30 08:55:20,201 - ERROR - Unexpected key 'day'
2023-04-30 08:55:20,202 - ERROR - Unexpected key 'time'
2023-04-30 08:55:20,202 - ERROR - Unexpected key 'open'
2023-04-30 08:55:20,203 - ERROR - Unexpected key 'day'
2023-04-30 08:55:20,204 - ERROR - Unexpected key 'time'
2023-04-30 08:55:20,204 - ERROR - Unexpected key 'close'
2023-04-30 08:55:20,206 - ERROR - Unexpected key 'day'
2023-04-30 08:55:20,207 - ERROR - Unexpec


Data retrieved from <firebase_admin.db.Reference object at 0x000001C606632150>

	Length of dataset: 275


2023-04-30 08:55:20,373 - ERROR - Invalid data type for key 'zip_code': Expected <class 'str'>, got <class 'int'>
2023-04-30 08:55:20,374 - ERROR - Invalid data type for key 'zip_code': Expected <class 'str'>, got <class 'int'>
2023-04-30 08:55:20,375 - ERROR - Invalid data type for key 'zip_code': Expected <class 'str'>, got <class 'int'>
2023-04-30 08:55:20,375 - ERROR - Invalid data type for key 'zip_code': Expected <class 'str'>, got <class 'int'>
2023-04-30 08:55:20,376 - ERROR - Invalid data type for key 'zip_code': Expected <class 'str'>, got <class 'int'>
2023-04-30 08:55:20,377 - ERROR - Unexpected key 'close'
2023-04-30 08:55:20,379 - ERROR - Unexpected key 'day'
2023-04-30 08:55:20,380 - ERROR - Unexpected key 'time'
2023-04-30 08:55:20,380 - ERROR - Unexpected key 'open'
2023-04-30 08:55:20,382 - ERROR - Unexpected key 'day'
2023-04-30 08:55:20,383 - ERROR - Unexpected key 'time'
2023-04-30 08:55:20,383 - ERROR - Unexpected key 'close'
2023-04-30 08:55:20,384 - ERROR - Unex